In [39]:
pip install -r CogModelingRNNsTutorial/requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'CogModelingRNNsTutorial/requirements.txt'


In [40]:
# Imports + defaults settings.
%load_ext autoreload
%autoreload 2

import haiku as hk
import jax
import jax.numpy as jnp
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import optax
import os
import warnings
import sys
import scipy
import pandas as pd

sys.path.insert(0, '.\CogModelingRNNsTutorial')

warnings.filterwarnings("ignore")

try:
    from google.colab import files
    _ON_COLAB = True
except:
    _ON_COLAB = False

import bandits
import disrnn
import hybrnn
import plotting
import subject_data
import rnn_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load data

In [41]:
mat = scipy.io.loadmat('..\data\data_for_Meike.mat')
print(mat)

data_arrays = mat['data'][0]

# Create a dictionary with keys as column names and values as arrays
data_dict = {
    'subject': data_arrays['subject'][0][0],
    'label': data_arrays['label'][0][0],
    'block': data_arrays['block'][0][0],
    'trial': data_arrays['trial'][0][0],
    'dtheta': data_arrays['dtheta'][0][0],
    'type': data_arrays['type'][0][0],
    'Pchoice': data_arrays['Pchoice'][0][0],
    'Preport': data_arrays['Preport'][0][0],
    'Pacc': data_arrays['Pacc'][0][0],
    'Schoice': data_arrays['Schoice'][0][0],
    'Sreport': data_arrays['Sreport'][0][0],
    'Sacc': data_arrays['Sacc'][0][0],
    'Srt1': data_arrays['Srt1'][0][0],
    'Srt2': data_arrays['Srt2'][0][0],
    'Spcorrect1': data_arrays['Spcorrect1'][0][0],
    'Spcorrect2': data_arrays['Spcorrect2'][0][0],
    'Snoise': data_arrays['Snoise'][0][0],
    'select': data_arrays['select'][0][0]
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data_dict)

# Display the DataFrame
print(df)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Wed Aug 14 12:02:52 2019', '__version__': '1.0', '__globals__': [], 'data': array([[(array([[ 1,  1,  1, ..., 30, 30, 30]], dtype=uint8), array([[42, 42, 42, ..., 71, 71, 71]], dtype=uint8), array([[1, 1, 1, ..., 4, 4, 4]], dtype=uint8), array([[  1,   2,   3, ..., 254, 255, 256]], dtype=uint16), array([[ 0.18, -0.11, -0.27, ...,  0.05,  0.05,  0.32]]), array([[2, 2, 1, ..., 2, 2, 1]], dtype=uint8), array([[-1, -1, -1, ..., -1,  1, -1]], dtype=int16), array([[5, 6, 3, ..., 3, 4, 1]], dtype=uint8), array([[0, 1, 1, ..., 0, 1, 0]], dtype=uint8), array([[ 1, -1, -1, ..., -1, -1,  1]], dtype=int16), array([[3, 4, 6, ..., 4, 3, 2]], dtype=uint8), array([[1, 1, 1, ..., 0, 0, 1]], dtype=uint8), array([[0.907, 0.38 , 0.388, ..., 0.308, 1.449, 0.568]]), array([[2576., 1483.,  940., ..., 1790., 1584.,  970.]]), array([[0.85821745, 0.67625963, 0.99      , ..., 0.51      , 0.59369021,
                0.62313369]]), array([[0.95393

Create input array and only keep the relevant columns

In [42]:
# (n_trials, n_blocks, #xs)
# coherence (dtheta), direction (?), Schoice, Sreport, RT (srt1), ID Partner (type), Pchoice, Preport, select 

# Select the relevant columns
selected_columns = ['subject', 'dtheta', 'type', 'Pchoice', 'Preport', 'Schoice', 'Sreport', 'Srt1', 'select']
selected_data = df[selected_columns].values

# Get the number of unique trials and blocks
num_trials = df['trial'].nunique()
num_blocks = df['block'].nunique()

# Create a dictionary to store data for each block
blocks_data = {}

# Iterate over each block and store the data
for block in range(1, num_blocks + 1):
    block_data = selected_data[df['block'] == block]
    blocks_data[block] = block_data

# Create the final reshaped array
xs = np.array([blocks_data[block] for block in range(1, num_blocks + 1)])

# Swap the dimensions to get the correct shape
xs = np.swapaxes(xs, 0, 1)

print(xs)

[[[ 1.     0.18   2.    ...  3.     0.907 -1.   ]
  [ 1.    -0.27   2.    ...  5.     0.29   1.   ]
  [ 1.     0.18   2.    ...  6.     0.415  1.   ]
  [ 1.    -0.18   1.    ...  2.     0.991  1.   ]]

 [[ 1.    -0.11   2.    ...  4.     0.38  -1.   ]
  [ 1.     0.27   1.    ...  4.     0.606 -1.   ]
  [ 1.     0.05   2.    ...  5.     0.278  1.   ]
  [ 1.     0.18   2.    ...  3.     0.322 -1.   ]]

 [[ 1.    -0.27   1.    ...  6.     0.388  1.   ]
  [ 1.    -0.27   2.    ...  2.     2.3   -1.   ]
  [ 1.    -0.11   1.    ...  5.     0.844  1.   ]
  [ 1.     0.05   1.    ...  3.     0.315 -1.   ]]

 ...

 [[30.     0.05   2.    ...  3.     0.536 -1.   ]
  [30.     0.32   1.    ...  3.     0.217 -1.   ]
  [30.    -0.13   1.    ...  2.     0.783 -1.   ]
  [30.     0.05   2.    ...  4.     0.308  1.   ]]

 [[30.    -0.13   1.    ...  1.     1.054 -1.   ]
  [30.    -0.21   2.    ...  3.     0.292  1.   ]
  [30.     0.05   1.    ...  3.     0.228 -1.   ]
  [30.     0.05   2.    ...  3.     

In [43]:
df['select'] # -1: partner
# TO DO: for creating a reward column:
# if select == 1: then reward = Sacc, if select == -1: reward = Pacc

0      -1
1      -1
2       1
3      -1
4      -1
       ..
7675    1
7676   -1
7677    1
7678   -1
7679    1
Name: select, Length: 7680, dtype: int16

In [44]:
# Display the reshaped data shape
print(xs.shape)

(1920, 4, 9)


Create input and target

In [45]:
# subject 1
xs_s1, ys_s1 = subject_data.load_data_for_one_subject(xs, 1)

print(xs_s1.shape)
print(ys_s1.shape)

(64, 4, 8)
(64, 4)


Create training and test set

In [46]:
train_s1, test_s1 = subject_data.format_into_datasets(xs_s1, ys_s1, rnn_utils.DatasetRNN)

# print(train_s1._xs.shape)
# print(train_s1._ys.shape)
# print(test_s1._xs.shape)
# print(test_s1._ys.shape)

print(train_s1._xs)
print(train_s1._ys)
print(test_s1._xs)
print(test_s1._ys)

[[[ 0.     0.     0.    ...  0.     0.     0.   ]
  [ 0.     0.     0.    ...  0.     0.     0.   ]]

 [[ 0.18   2.    -1.    ...  3.     0.907 -1.   ]
  [ 0.18   2.    -1.    ...  6.     0.415  1.   ]]

 [[-0.11   2.    -1.    ...  4.     0.38  -1.   ]
  [ 0.05   2.    -1.    ...  5.     0.278  1.   ]]

 ...

 [[ 0.05   2.     1.    ...  2.     0.62  -1.   ]
  [-0.11   1.     1.    ...  4.     0.71   1.   ]]

 [[ 0.18   1.     1.    ...  4.     0.581  1.   ]
  [-0.11   2.    -1.    ...  3.     0.525 -1.   ]]

 [[-0.11   2.    -1.    ...  1.     0.815 -1.   ]
  [-0.05   1.     1.    ...  2.     0.463 -1.   ]]]
[[ 3.  6.]
 [ 4.  5.]
 [ 6.  5.]
 [ 5.  4.]
 [ 1.  6.]
 [ 4.  2.]
 [ 2.  1.]
 [ 4.  2.]
 [ 1.  6.]
 [ 6.  6.]
 [ 2.  6.]
 [ 6.  3.]
 [ 1.  4.]
 [ 1.  6.]
 [ 1.  5.]
 [ 4.  3.]
 [ 4.  5.]
 [ 1.  5.]
 [ 2.  1.]
 [ 2.  1.]
 [ 3.  4.]
 [ 1.  1.]
 [ 2.  6.]
 [ 3.  6.]
 [ 6.  4.]
 [ 5.  5.]
 [ 3.  4.]
 [ 1.  6.]
 [ 4.  6.]
 [ 6.  5.]
 [ 5.  5.]
 [ 3.  5.]
 [ 6.  1.]
 [ 4.  3.]
 [ 5.  5

## Set up the Disentangled RNN

In [37]:
# Number of latent units in the model.
latent_size = 5  #@param

# Number of hidden units in each of the two layers of the update MLP.
update_mlp_shape = (3,3,)  #@param

# Number of hidden units in each of the two layers of the choice MLP.
choice_mlp_shape = (2,)

def make_disrnn():
  model = disrnn.HkDisRNN(latent_size = latent_size,
                          update_mlp_shape = update_mlp_shape,
                          choice_mlp_shape = choice_mlp_shape,
                          target_size=2)
  return model

def make_disrnn_eval():
  model = disrnn.HkDisRNN(latent_size = latent_size,
                          update_mlp_shape = update_mlp_shape,
                          choice_mlp_shape = choice_mlp_shape,
                          target_size=2,
                          eval_mode=True)
  return model


optimizer = optax.adam(learning_rate=1e-2)

Fit disRNN with no penalty at first, to get good quality-of-fit

In [53]:
# You can experiment with different values, but colab has been tested with 1000.
print(('Warning: this step can be rather time consuming without GPU access. If you are not running on a GPU\n, '
       'you may want to set n_steps to a very low value and return to the exercise when you \n'
       'have access to hardware acceleration.'))
n_steps = 1000 #@param
information_penalty = 0

disrnn_params, opt_state, losses = rnn_utils.train_model(
    model_fun = make_disrnn,
    dataset = train_s1, 
    optimizer = optimizer,
    loss_fun = 'penalized_categorical',
    penalty_scale=information_penalty,
    n_steps=n_steps,
    do_plot=False,
    truncate_seq_length=200,
)

, you may want to set n_steps to a very low value and return to the exercise when you 
have access to hardware acceleration.


ValueError: Incompatible shapes for broadcasting: shapes=[(2, 13, 1), (7, 5)]

Now fit more steps with a penalty, to encourage it to find a simple solution

In [ ]:
# You can experiment with different values, but colab has been tested with 3000.
n_steps = 3000  #@param
information_penalty = 1e-3  #@param

disrnn_params, opt_state, losses = rnn_utils.train_model(
    model_fun = make_disrnn,
    dataset = dataset_train, # TO DO
    optimizer = optimizer,
    loss_fun = 'penalized_categorical',
    params=disrnn_params,
    opt_state=opt_state,
    penalty_scale=information_penalty,
    n_steps=n_steps,
    truncate_seq_length=200,
)

Visualize model latents
Visualize bottleneck latents + learned update.

In [ ]:
disrnn.plot_bottlenecks(disrnn_params)
disrnn.plot_update_rules(disrnn_params, make_disrnn_eval)